## Instalacion de todas las dependencias necesarias

In [1]:
! pip install wget

## Limpieza de ficheros
Se eliminan los posibles ficheros de texto descargados previamente

In [2]:
! rm -r *.txt

# Descarga del fichero en cuestion

In [3]:
import wget
url = "https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt"
fichero = wget.download(url)
# Se muestra el nombre del fichero
fichero

'el_quijote.txt'

# Procesado fichero
Primero se va a cargar el fichero,después se va a separar línea por línea, y por último se van a eliminar las comas, separadores de línea y los puntos

In [4]:
file = open("el_quijote.txt")
lineas = file.readlines()

lineas_finales = []

for linea in lineas:
    # Sustitución de la separación de líneas
    linea_provisional = linea.replace("\n","")
    # Sustitución de los puntos
    linea_provisional = linea_provisional.replace("."," ")
    # Sustitución de las comas
    linea_provisional = linea_provisional.replace(",","")
    # Otros carateres raros encontrados
    linea_provisional = linea_provisional.replace("!","")
    linea_provisional = linea_provisional.replace("-","")
    linea_provisional = linea_provisional.replace(":","")
    linea_provisional = linea_provisional.replace("?","")
   
    # Se coloca en la lista final
    lineas_finales.append(linea_provisional)

In [5]:
print(lineas_finales)

['DON QUIJOTE DE LA MANCHA', 'Miguel de Cervantes Saavedra', '', 'PRIMERA PARTE', 'CAPÍTULO 1 Que trata de la condición y ejercicio del famoso hidalgo D  Quijote de la Mancha', 'En un lugar de la Mancha de cuyo nombre no quiero acordarme no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero adarga antigua rocín flaco y galgo corredor  Una olla de algo más vaca que carnero salpicón las más noches duelos y quebrantos los sábados lentejas los viernes algún palomino de añadidura los domingos consumían las tres partes de su hacienda  El resto della concluían sayo de velarte calzas de velludo para las fiestas con sus pantuflos de lo mismo los días de entre semana se honraba con su vellori de lo más fino  Tenía en su casa una ama que pasaba de los cuarenta y una sobrina que no llegaba a los veinte y un mozo de campo y plaza que así ensillaba el rocín como tomaba la podadera  Frisaba la edad de nuestro hidalgo con los cincuenta años era de complexión recia sec

# MapReduce
En este apartado de la práctica se va a implementar el algoritmo de mapreduce
## Preparación entorno
Se importan todas las librerías necesarias, tanto para el tipado como para el algoritmo

In [6]:
import functools
from typing import Tuple, Set

## Creación función map
Esta función va a recibir una lista de strings y va a agrupar los strings por longitud en un diccionario

In [7]:
def funcion_map(string_entrada:str)->Tuple[int,Set[str]]:
    longitudes_added = []
    tupla_final = tuple()
    lista_entrada = string_entrada.split()
    
    # Itera la lista recibida
    for elemento in lista_entrada:
        
        # Comprueba que o exista una clave con este valor
        if len(elemento) not in longitudes_added:
            # En el caso de que no se haya añadido ningún elemento de esa longitud se añade a la lista
            lista_tupla = list(tupla_final)
            lista_tupla.append(len(elemento))
            lista_tupla.append({elemento})
            longitudes_added.append(len(elemento))
            tupla_final = tuple(lista_tupla)  
        else:
            lista_tupla = list(tupla_final)
            # Se saca el indice donde se encuentra esa clave
            indice = tupla_final.index(len(elemento))
            set_previo = lista_tupla[indice+1]
            set_nuevo = set_previo.add(elemento)
            tupla_final = tuple(lista_tupla)
    return tupla_final


In [8]:
funcion_map('DON QUIJOTE DE LA MANCHA ')

(3, {'DON'}, 7, {'QUIJOTE'}, 2, {'DE', 'LA'}, 6, {'MANCHA'})

## Ejecución función map
En este apartado se va a pasar a la funcionalidad map la función que se ha creado

In [9]:
resultado_map =tuple( map(lambda linea:funcion_map(linea),lineas_finales))
print(resultado_map)

((3, {'DON'}, 7, {'QUIJOTE'}, 2, {'LA', 'DE'}, 6, {'MANCHA'}), (6, {'Miguel'}, 2, {'de'}, 9, {'Cervantes'}, 8, {'Saavedra'}), (), (7, {'PRIMERA'}, 5, {'PARTE'}), (9, {'ejercicio', 'CAPÍTULO'}, 1, {'D', '1', 'y'}, 3, {'Que', 'del'}, 5, {'trata'}, 2, {'de', 'la'}, 10, {'condición'}, 6, {'famoso', 'Mancha'}, 7, {'hidalgo', 'Quijote'}), (2, {'me', 'al', 'El', 'os', 'no', 'el', 'No', 'mi', 'un', 'su', 'le', 'ha', 'en', 'la', 'En', 'si', 'lo', 'se', 'Es', 'de', 'ni'}, 5, {'della', 'estas', 'donde', 'tanta', 'darle', 'año)', 'galgo', 'otros', 'tener', 'decir', 'lanza', 'altos', 'autor', 'quejo', 'lleno', 'Silva', 'hacen', 'letra', 'deste', 'haber', 'recia', 'deseo', 'lugar', 'salga', 'leer;', 'allí', 'veces', 'punto', 'mucho', 'sólo', 'entre', 'días', 'pobre', 'tomar', 'pluma', 'saber', 'llama', 'resto', 'leía', 'aquel', 'prosa', 'libro', 'suyas', 'mismo', 'campo', 'flaco', 'años', 'amigo', 'gusto', 'tanto', 'basta', 'ratos', 'todos', 'plaza'}, 6, {'juicio', 'cartas', 'merece', 'tení

## Función reduce
Esta función va a recibir la lista resultado de ejecutar la función map, y para cada par de tuplas que reciba va a iterar en el número de letras y unir las dos listas correspondientes a los números que coincidan

In [10]:
def funcion_reduce(tupla_a:Tuple[int,Set[str]],tupla_b:Tuple[int,Set[str]])->Tuple[int,Set[str]]:
    
    numeros_a = []
    numeros_b = []
    numeros_coincidentes = []
    numeros_no_coincidentes_a = []
    numeros_no_coincidentes_b = []
    tupla_final = tuple()
    
    # Itera en la tupla a para averigurar que longitudes contiene
    lista_tupla_a  = list(tupla_a)
    for elemento_a in lista_tupla_a:
        if type(elemento_a)==int:
            numeros_a.append(elemento_a)
    # Lo mismo para
    lista_tupla_b  = list(tupla_b)
    for elemento_b in lista_tupla_b:
        if type(elemento_b)==int:
            numeros_b.append(elemento_b)
                
    # Itera en las listas de elementos para ver cuales coinciden 
    for longitud in numeros_a:
        if longitud in numeros_b:
            numeros_coincidentes.append(longitud)
        else:
            numeros_no_coincidentes_a.append(longitud)
    # Ahora que ya se tienen los coincidentes de a y b, y los no coincidentes de a , queda comprobar los de b
    for longitud in numeros_b:
        if longitud not in numeros_coincidentes:
            numeros_no_coincidentes_b.append(longitud)

            
    # Se añaden primero los números no coincidentes de a
    for longitud in numeros_no_coincidentes_a:
        indice = lista_tupla_a.index(longitud)
        elemento = lista_tupla_a[indice+1]
        lista_tupla = list(tupla_final)
        lista_tupla.append(longitud)
        lista_tupla.append(elemento)
        tupla_final = tuple(lista_tupla)
        

    # No coincidentes de b
    for longitud in numeros_no_coincidentes_b:
        indice = lista_tupla_b.index(longitud)
        elemento = lista_tupla_b[indice+1]
        lista_tupla = list(tupla_final)
        lista_tupla.append(longitud)
        lista_tupla.append(elemento)
        tupla_final = tuple(lista_tupla)
        
 
    # Coincidentes de a y b
    for longitud in numeros_coincidentes:
        indice_a = lista_tupla_a.index(longitud)
        indice_b = lista_tupla_b.index(longitud)
        
        set_a = lista_tupla_a[indice_a+1]
        set_b = lista_tupla_b[indice_b+1]
        set_conjunto = set_a
        lista_set_b = list(set_b)
        
        # Se itera por cada elemento  del set b para añadirlo al set a, update no funciona
        for i in range(0,len(lista_set_b)):
            set_conjunto.add(lista_set_b[i])  
        
        lista_tupla = list(tupla_final)
        lista_tupla.append(longitud)
        lista_tupla.append(set_conjunto)
        tupla_final = tuple(lista_tupla)
    return tupla_final


### Ejecución de prueba
Para ver si el resultado obtenido se corresponde con el resultado buscado

In [11]:
list(funcion_reduce(resultado_map[0],resultado_map[1]))

[3,
 {'DON'},
 7,
 {'QUIJOTE'},
 9,
 {'Cervantes'},
 8,
 {'Saavedra'},
 2,
 {'DE', 'LA', 'de'},
 6,
 {'MANCHA', 'Miguel'}]

## Ejecución final
Se llama a la función reduce con la función creada anteriormente

In [12]:
tuple( functools.reduce(lambda tupla_a,tupla_b:funcion_reduce(tupla_a,tupla_b),resultado_map))

(18,
 {'convirtiéndoseles', 'estrechísimamente'},
 17,
 {'Agredeciéronselo', 'cortesísimamente'},
 19,
 {'extraordinariamente'},
 16,
 {'Atentísimamente',
  'agradeciéndoles',
  'alternativamente',
  'certificándoles',
  'congratulándose',
  'desabrochándole',
  'descansadamente;',
  'describiéndolos',
  'desembarcásemos',
  'desenfadadamente',
  'desenvueltamente',
  'encomendándonos',
  'entretenimiento;',
  'entretenimientos',
  'escudriñáronse',
  'facilísimamente',
  'inadvertidamente',
  'malbaratándolas',
  'maravillosamente',
  'precipitadamente',
  'quebrantamientos',
  'revolviéndosele',
  'transformaciones'},
 15,
 {'Pintiquiniestra',
  'Prometímosselo',
  'Respondiéronme',
  'acabándonoslas',
  'acompañamiento',
  'acompañáramos',
  'acontecimiento;',
  'acontecimientos',
  'acrecentóseles',
  'administración',
  'afirmativamente',
  'agradecéroslas',
  'agradeciéndole',
  'aguardándolos;',
  'aplicándoselas',
  'aprovechamiento',
  'aprovechán